In [1]:
import dspy

gpt4o_mini = dspy.LM('openai/gpt-4o-mini', max_tokens=2000)
gpt4o = dspy.LM('openai/gpt-4o', max_tokens=2000)
dspy.configure(lm=gpt4o_mini)  # we'll use gpt-4o-mini as the default LM, unless otherwise specified

In [2]:
from dspy.datasets import MATH

dataset = MATH(subset='algebra')
print(len(dataset.train), len(dataset.dev))

C:\Users\marci\anaconda3\envs\DATAENLIGHT_AI_LAB\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\marci\anaconda3\envs\DATAENLIGHT_AI_LAB\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marci\.cache\huggingface\hub\datasets--DigitalLearningGmbH--MATH-lighteval. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an

350 350


In [4]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)


Question: The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.
Answer: 221


In [5]:
module = dspy.ChainOfThought("question -> answer")
module(question=example.question)

Prediction(
    reasoning="To calculate Cal O'Ree's expected weight over the ten weeks, we will start with his initial weight and reduce it by $1\\%$ each week. \n\n1. Let \\( W_0 \\) be his initial weight, which is 244 pounds.\n2. The weight at the end of each week can be determined by the formula:\n\n   \\[\n   W_n = W_{n-1} \\times (1 - 0.01)\n   \\]\n\n   Simplifying this, we can express it as:\n\n   \\[\n   W_n = W_{n-1} \\times 0.99\n   \\]\n\n3. We can repeat this process for nine weekly reductions. Therefore, after ten weeks, we can express the final weight \\( W_{10} \\) as:\n\n   \\[\n   W_{10} = W_0 \\times (0.99)^{10}\n   \\]\n\n4. Now substituting \\( W_0 = 244 \\):\n\n   \\[\n   W_{10} = 244 \\times (0.99)^{10}\n   \\]\n\n5. Calculating \\( (0.99)^{10} \\):\n\n   \\[\n   (0.99)^{10} \\approx 0.9044\n   \\]\n\n6. Therefore,\n\n   \\[\n   W_{10} \\approx 244 \\times 0.9044 \\approx 220.4786\n   \\]\n\n7. Rounding to the nearest whole number, we find:\n\n   \\[\n   W_{10} \\

In [6]:
THREADS = 24
kwargs = dict(num_threads=THREADS, display_progress=True, display_table=5)
evaluate = dspy.Evaluate(devset=dataset.dev, metric=dataset.metric, **kwargs)

evaluate(module)

Average Metric: 253.00 / 350 (72.3%): 100%|██████████| 350/350 [02:06<00:00,  2.76it/s]

2025/12/18 21:33:14 INFO dspy.evaluate.evaluate: Average Metric: 253 / 350 (72.3%)


,question,example_reasoning,example_answer,pred_reasoning,pred_answer,method
0,What is the smallest integer value of $c$ such that the function $...,The given function has a domain of all real numbers if and only if...,1,For the function \( f(x) = \frac{x^2 + 1}{x^2 - x + c} \) to have ...,1,✔️ [True]
1,What is the least value of $x$ that is a solution of $|{-x+3}|=7$?,"In order to have $|{-x+3}| = 7$, we must have $-x + 3 = 7$ or $-x ...",-4,"To solve the equation \( |{-x+3}|=7 \), we need to consider the tw...",-4,✔️ [True]
2,Evaluate $\left\lceil -\frac{7}{4}\right\rceil$.,"$-\frac{7}{4}$ is between $-1$ and $-2$, so $\left\lceil -\frac{7}...",-1,"To evaluate \(\left\lceil -\frac{7}{4}\right\rceil\), we first nee...",-1,✔️ [True]
3,"A triangle has vertices at coordinates $(11,1)$, $(2,3)$ and $(3,7...",We must find the distance between each pair of points by using the...,10,To find the length of the longest side of the triangle with vertic...,10,✔️ [True]
4,Let $f(x) = x + 2$ and $g(x) = 1/f(x)$. What is $g(f(-3))$?,"First, we find that $f(-3) = (-3) + 2 = -1$. Then, $$g(f(-3)) = g(...",1,"First, we need to evaluate \( f(-3) \) using the function \( f(x) ...",1,✔️ [True]


EvaluationResult(score=72.29, results=<list of 350 results>)

In [8]:
kwargs = dict(num_threads=THREADS, teacher_settings=dict(lm=gpt4o), prompt_model=gpt4o_mini)
optimizer = dspy.MIPROv2(metric=dataset.metric, auto="medium", **kwargs)

kwargs = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
optimized_module = optimizer.compile(module, trainset=dataset.train, **kwargs)

2025/12/18 21:35:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 280

2025/12/18 21:35:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/12/18 21:35:44 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/12/18 21:35:44 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


  6%|▌         | 4/70 [00:21<05:53,  5.36s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/12


  6%|▌         | 4/70 [00:15<04:21,  3.97s/it]


...

2025/12/18 21:52:07 INFO dspy.evaluate.evaluate: Average Metric: 28 / 35 (80.0%)
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 80.0 on minibatch of size 35 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 11'].
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [88.57, 80.0, 94.29, 97.14, 82.86, 80.0, 85.71, 80.0, 82.86, 65.71, 80.0, 77.14, 82.86, 88.57, 91.43, 80.0, 80.0]
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.36, 77.14, 85.71, 86.43]
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.43
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 22 / 23 - Minibatch ==



Average Metric: 30.00 / 35 (85.7%): 100%|██████████| 35/35 [00:00<00:00, 510.58it/s] 

2025/12/18 21:52:07 INFO dspy.evaluate.evaluate: Average Metric: 30 / 35 (85.7%)
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 85.71 on minibatch of size 35 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 8'].
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [88.57, 80.0, 94.29, 97.14, 82.86, 80.0, 85.71, 80.0, 82.86, 65.71, 80.0, 77.14, 82.86, 88.57, 91.43, 80.0, 80.0, 85.71]
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.36, 77.14, 85.71, 86.43]
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.43
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 23 / 23 - Full Evaluation =====
2025/12/18 21:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score:


Average Metric: 241.00 / 280 (86.1%): 100%|██████████| 280/280 [01:36<00:00,  2.91it/s]

2025/12/18 21:53:43 INFO dspy.evaluate.evaluate: Average Metric: 241 / 280 (86.1%)
2025/12/18 21:53:43 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.36, 77.14, 85.71, 86.43, 86.07]
2025/12/18 21:53:43 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.43
2025/12/18 21:53:43 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/12/18 21:53:43 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/12/18 21:53:43 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 86.43!


In [9]:
evaluate(optimized_module)

Average Metric: 306.00 / 350 (87.4%): 100%|██████████| 350/350 [02:18<00:00,  2.52it/s]

2025/12/18 21:59:45 INFO dspy.evaluate.evaluate: Average Metric: 306 / 350 (87.4%)


,question,example_reasoning,example_answer,pred_reasoning,pred_answer,method
0,What is the smallest integer value of $c$ such that the function $...,The given function has a domain of all real numbers if and only if...,1,To determine the smallest integer value of \( c \) such that the f...,1,✔️ [True]
1,What is the least value of $x$ that is a solution of $|{-x+3}|=7$?,"In order to have $|{-x+3}| = 7$, we must have $-x + 3 = 7$ or $-x ...",-4,The equation given is \( |{-x+3}|=7 \). To solve this absolute val...,-4,✔️ [True]
2,Evaluate $\left\lceil -\frac{7}{4}\right\rceil$.,"$-\frac{7}{4}$ is between $-1$ and $-2$, so $\left\lceil -\frac{7}...",-1,"To evaluate \(\left\lceil -\frac{7}{4}\right\rceil\), we first nee...",-1,✔️ [True]
3,"A triangle has vertices at coordinates $(11,1)$, $(2,3)$ and $(3,7...",We must find the distance between each pair of points by using the...,10,To find the length of the sides of the triangle with vertices at \...,10,✔️ [True]
4,Let $f(x) = x + 2$ and $g(x) = 1/f(x)$. What is $g(f(-3))$?,"First, we find that $f(-3) = (-3) + 2 = -1$. Then, $$g(f(-3)) = g(...",1,"To solve for \( g(f(-3)) \), we first need to calculate \( f(-3) \...",1,✔️ [True]


EvaluationResult(score=87.43, results=<list of 350 results>)